In [2]:
# -----------------------------------------
# INSTALL REQUIRED LIBRARIES (Google Colab)
# -----------------------------------------
!pip install -q transformers datasets sentence-transformers hnswlib accelerate bitsandbytes peft

# -----------------------------------------
# IMPORT LIBRARIES
# -----------------------------------------
from google.colab import drive
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sentence_transformers import SentenceTransformer
import hnswlib
import numpy as np
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
)
from peft import get_peft_model, LoraConfig, TaskType
import torch
import os

# -----------------------------------------
# MOUNT GOOGLE DRIVE
# -----------------------------------------
drive.mount('/content/drive')

# -----------------------------------------
# STEP 1: UPLOAD AND LOAD DATASET
# -----------------------------------------
from google.colab import files
uploaded = files.upload()
df = pd.read_csv("ARS_DATA.csv")

# -----------------------------------------
# STEP 2: CLEANING AND ENCODING
# -----------------------------------------
def clean_and_encode_data(df):
    df.dropna(inplace=True)
    label_cols = ['Gender', 'Dietary Preferences', 'Fitness Goal', 'Exercise Level', 'Meal Name', 'Meal Time']
    encoders = {col: LabelEncoder() for col in label_cols}
    for col in label_cols:
        df[col + '_original'] = df[col]
        df[col] = encoders[col].fit_transform(df[col])
    scaler = StandardScaler()
    df[['Height (cm)', 'Weight (kg)', 'BMI']] = scaler.fit_transform(df[['Height (cm)', 'Weight (kg)', 'BMI']])
    return df, encoders, scaler

df, encoders, scaler = clean_and_encode_data(df)

# -----------------------------------------
# STEP 3: CREATE HUMAN-READABLE DESCRIPTIONS
# -----------------------------------------
def create_description(df):
    df['Full_Description'] = df.apply(lambda row:
        f"{row['Gender_original']}, {row['Age']} years old, {row['Height (cm)']:.2f} cm, {row['Weight (kg)']:.2f} kg, "
        f"{row['Dietary Preferences_original']} diet, {row['Fitness Goal_original']}, "
        f"{row['Exercise Level_original']} exercise, BMI {row['BMI']:.2f}",
        axis=1)
    return df

df = create_description(df)

# -----------------------------------------
# STEP 4: GENERATE EMBEDDINGS
# -----------------------------------------
embedder = SentenceTransformer('paraphrase-MiniLM-L3-v2')
embeddings = embedder.encode(df['Full_Description'].tolist(), convert_to_numpy=True)

# -----------------------------------------
# STEP 5: BUILD HNSW INDEX
# -----------------------------------------
def build_hnsw_index(embeddings, ef=100, M=16):
    dim = embeddings.shape[1]
    index = hnswlib.Index(space='cosine', dim=dim)
    index.init_index(max_elements=len(embeddings), ef_construction=ef, M=M)
    index.add_items(embeddings, ids=list(range(len(embeddings))))
    index.set_ef(ef)
    return index

index = build_hnsw_index(embeddings)

# -----------------------------------------
# STEP 6: PREPARE DATA FOR GPT FINE-TUNING
# -----------------------------------------
def prepare_finetuning_data(df):
    df['input'] = df['Full_Description'].apply(lambda x: f"{x} => Recommended meal and workout:")
    df['output'] = df['Meal Name_original'].astype(str)
    df['text'] = df['input'] + " " + df['output']
    return Dataset.from_pandas(df[['text']])

ds = prepare_finetuning_data(df)

# -----------------------------------------
# STEP 7: LOAD MODEL AND APPLY LoRA
# -----------------------------------------
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

quant_config = BitsAndBytesConfig(load_in_8bit=True)
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto"
)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(base_model, lora_config)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# -----------------------------------------
# STEP 8: TOKENIZE DATA
# -----------------------------------------
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=64)

tokenized_dataset = ds.map(tokenize_function, batched=True)
small_ds = tokenized_dataset.select(range(min(7000, len(tokenized_dataset))))

# -----------------------------------------
# STEP 9: TRAINING
# -----------------------------------------
save_dir = "/content/drive/MyDrive/deepseek"
os.makedirs(save_dir, exist_ok=True)

training_args = TrainingArguments(
    output_dir=save_dir,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_dir=os.path.join(save_dir, 'logs'),
    report_to="none",
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_ds,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

# -----------------------------------------
# STEP 10: SAVE ALL ARTIFACTS
# -----------------------------------------
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
df.to_csv(os.path.join(save_dir, "ARS_df.csv"), index=False)
np.save(os.path.join(save_dir, "ARS_embeddings.npy"), embeddings)
index.save_index(os.path.join(save_dir, "ARS_hnsw_index.bin"))
print(f"All artifacts saved to {save_dir}")


Mounted at /content/drive


Saving ARS_DATA.csv to ARS_DATA.csv


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

<ipython-input-2-a488eb9ca8f4>:153: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


All artifacts saved to /content/drive/MyDrive/deepseek


In [3]:
# -----------------------------------------
# INSTALL INFERENCE DEPENDENCIES
# -----------------------------------------
!pip install -q transformers sentence-transformers hnswlib

# -----------------------------------------
# IMPORT LIBRARIES & MOUNT DRIVE
# -----------------------------------------
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import hnswlib
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# -----------------------------------------
# LOAD SAVED ARTIFACTS
# -----------------------------------------
base_path = "/content/drive/MyDrive/deepseek"

df = pd.read_csv(f"{base_path}/ARS_df.csv")
embeddings = np.load(f"{base_path}/ARS_embeddings.npy")

dim = embeddings.shape[1]
index = hnswlib.Index(space='cosine', dim=dim)
index.load_index(f"{base_path}/ARS_hnsw_index.bin")
index.set_ef(100)

tokenizer = AutoTokenizer.from_pretrained(base_path)
model = AutoModelForCausalLM.from_pretrained(base_path).to("cuda")
gen_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

embedder = SentenceTransformer('paraphrase-MiniLM-L3-v2')

# -----------------------------------------
# RAG-BASED RECOMMEND FUNCTION
# -----------------------------------------
def recommend(query, embedder, index, df, gen_pipeline, top_k=3):
    query_vec = embedder.encode([query])
    ids, distances = index.knn_query(query_vec, k=top_k)
    similar_profiles = df.iloc[ids[0]]['Full_Description'].tolist()

    context = "\n".join([f"- {desc}" for desc in similar_profiles])
    prompt = (
        f"You are a fitness and nutrition expert.\n"
        f"Here are descriptions of similar users:\n{context}\n\n"
        f"Now respond to this query:\n{query} => Recommended meal and workout:"
    )

    response = gen_pipeline(prompt, max_new_tokens=80, do_sample=True, temperature=0.7)[0]['generated_text']
    return response

# -----------------------------------------
# EXAMPLE QUERIES
# -----------------------------------------
query1 = "Male, 30 years old, 160 cm, 80 kg, vegetarian diet, weight loss, moderate exercise, BMI 23"
print("\n--- Recommendation for Query 1 ---\n")
print(recommend(query1, embedder, index, df, gen_pipeline))

query2 = "Female, 40 years old, 155 cm, 70 kg, keto diet, muscle gain, intense exercise, BMI 26"
print("\n--- Recommendation for Query 2 ---\n")
print(recommend(query2, embedder, index, df, gen_pipeline))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Device set to use cuda:0



--- Recommendation for Query 1 ---

You are a fitness and nutrition expert.
Here are descriptions of similar users:
- Male, 35 years old, 1.23 cm, 0.11 kg, Vegetarian diet, Fat Loss, Moderate (3x/week) exercise, BMI -0.60
- Male, 64 years old, -0.40 cm, 1.26 kg, Vegetarian diet, Fat Loss, Moderate (3x/week) exercise, BMI 1.23
- Male, 23 years old, 0.89 cm, -1.37 kg, Vegetarian diet, Fat Loss, High (5x/week) exercise, BMI -1.45

Now respond to this query:
Male, 30 years old, 160 cm, 80 kg, vegetarian diet, weight loss, moderate exercise, BMI 23 => Recommended meal and workout: Chickpea Curry (Protein) + Chickpea Propping (Protein) + Chickpea Propping (Protein) + Chickpea Propping (Protein) + Chickpea Propping (Protein) + Chickpea Propping (Protein) + Chickpea Propping (Protein) + Chickpea Propping (Protein) + Chick

--- Recommendation for Query 2 ---

You are a fitness and nutrition expert.
Here are descriptions of similar users:
- Female, 54 years old, 1.64 cm, 2.00 kg, Keto diet, Mus

In [1]:
# -----------------------------------------
# INSTALL DEPENDENCIES
# -----------------------------------------
!pip install -q transformers sentence-transformers hnswlib

# -----------------------------------------
# IMPORT LIBRARIES & MOUNT DRIVE
# -----------------------------------------
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import hnswlib
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# -----------------------------------------
# LOAD SAVED ARTIFACTS FROM GOOGLE DRIVE
# -----------------------------------------
base_path = "/content/drive/MyDrive/deepseek"

df = pd.read_csv(f"{base_path}/ARS_df.csv")
embeddings = np.load(f"{base_path}/ARS_embeddings.npy")

dim = embeddings.shape[1]
index = hnswlib.Index(space='cosine', dim=dim)
index.load_index(f"{base_path}/ARS_hnsw_index.bin")
index.set_ef(100)

tokenizer = AutoTokenizer.from_pretrained(base_path)
model = AutoModelForCausalLM.from_pretrained(base_path).to("cuda")
gen_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

embedder = SentenceTransformer('paraphrase-MiniLM-L3-v2')

# -----------------------------------------
# RAG-BASED RECOMMEND FUNCTION
# -----------------------------------------
def recommend(query, embedder, index, df, gen_pipeline, top_k=3):
    query_vec = embedder.encode([query])
    ids, distances = index.knn_query(query_vec, k=top_k)
    similar_profiles = df.iloc[ids[0]]['Full_Description'].tolist()

    context = "\n".join([f"- {desc}" for desc in similar_profiles])
    prompt = (
        f"You are a fitness and nutrition expert.\n"
        f"Here are descriptions of similar users:\n{context}\n\n"
        f"Now respond to this query:\n{query} => Recommended meal and workout:"
    )

    response = gen_pipeline(prompt, max_new_tokens=80, do_sample=True, temperature=0.7)[0]['generated_text']
    return response

# -----------------------------------------
# EXAMPLE QUERIES
# -----------------------------------------
query1 = "Male, 30 years old, 160 cm, 80 kg, vegetarian diet, weight loss, moderate exercise, BMI 23"
print("\n--- Recommendation for Query 1 ---\n")
print(recommend(query1, embedder, index, df, gen_pipeline))

query2 = "Female, 40 years old, 155 cm, 70 kg, keto diet, muscle gain, intense exercise, BMI 26"
print("\n--- Recommendation for Query 2 ---\n")
print(recommend(query2, embedder, index, df, gen_pipeline))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Device set to use cuda:0



--- Recommendation for Query 1 ---

You are a fitness and nutrition expert.
Here are descriptions of similar users:
- Male, 35 years old, 1.23 cm, 0.11 kg, Vegetarian diet, Fat Loss, Moderate (3x/week) exercise, BMI -0.60
- Male, 64 years old, -0.40 cm, 1.26 kg, Vegetarian diet, Fat Loss, Moderate (3x/week) exercise, BMI 1.23
- Male, 23 years old, 0.89 cm, -1.37 kg, Vegetarian diet, Fat Loss, High (5x/week) exercise, BMI -1.45

Now respond to this query:
Male, 30 years old, 160 cm, 80 kg, vegetarian diet, weight loss, moderate exercise, BMI 23 => Recommended meal and workout: Chickpea Curry (Protein) + Chickpea Propping (Protein) + Muscle道 (Protein) + Weight pull (Protein) + Chickpea Propping (Protein) + Rest (Protein) + Rest (Protein)
Your recommendation: Protein Matrix
Why? Because it provides essential amino acids, which support muscle growth and recovery. The varied protein sources

--- Recommendation for Query 2 ---

You are a fitness and nutrition expert.
Here are descriptions o